# Stanford CoreNLP Exploitation Unambiguous

In [1]:
import random
import pickle

import numpy as np

import stanza
from stanza.server import CoreNLPClient
stanza.download('en') 
import sys, os


# Add neural coref to SpaCy's pipe
nlp = stanza.Pipeline()
doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
doc.sentences[0].print_dependencies()
pass




2020-08-26 09:00:14 INFO: Downloading default packages for language: en (English)...
2020-08-26 09:00:15 INFO: File exists: /Users/sakshiudeshi/stanza_resources/en/default.zip.
2020-08-26 09:00:18 INFO: Finished downloading models and saved to /Users/sakshiudeshi/stanza_resources.
2020-08-26 09:00:18 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-08-26 09:00:18 INFO: Use device: cpu
2020-08-26 09:00:18 INFO: Loading: tokenize
2020-08-26 09:00:18 INFO: Loading: pos
2020-08-26 09:00:19 INFO: Loading: lemma
2020-08-26 09:00:19 INFO: Loading: depparse
2020-08-26 09:00:20 INFO: Loading: ner
2020-08-26 09:00:20 INFO: Done loading processors!


('Barack', '4', 'nsubj:pass')
('Obama', '1', 'flat')
('was', '4', 'aux:pass')
('born', '0', 'root')
('in', '6', 'case')
('Hawaii', '4', 'obl')
('.', '4', 'punct')


In [2]:
os.environ["CORENLP_HOME"] = "/Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0"
text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
with CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'],
        timeout=30000,
        memory='16G') as client:
    ann = client.annotate(text)

Starting server with command: java -Xmx16G -cp /Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-138b6859faa54670.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref


In [3]:
verb_list_p1 = ['asked', 'disclosed to', 'told'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified', 'spoke with'] ##  pronoun2 = ['his', 'her', 'their']

In [4]:
def choose_pronoun_type (verb):
    return ['He', 'She', 'They']

In [5]:
filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "about "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [6]:
verb_action = {}

action1 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p1[1]] = [action1] ### disclosed to
verb_action[verb_list_p1[2]] = [action1] ### told

verb_action[verb_list_p2[0]] = [action1] ### notified

In [7]:
### spoke with 
### asked
action2 = ['painting', 'dancing',
           'fencing', 'gymnastics',
           'independent cinema', 'woodworking',
           'studying', 'horse racing',
           'singing', 'kayaking', 'football',
           'baseball', 'basketball', 'quizzing', 
           'gardening', 'blogging', 'board games', 
           'breadmaking', 'baking']
verb_action[verb_list_p2[1]] = [action2]
verb_action[verb_list_p1[0]] = [action2]

In [8]:
auxiliary_verb = ['standing', 'speaking', 'talking', 
                  'passing by', 'sitting', 'sitting on the chair']

In [9]:
def predict_clusters(sentence):
    doc = client.annotate(sentence)
#     print((doc.))
    return(doc.corefChain)
#     if doc._.has_coref: 
#         return (doc._.coref_resolved, doc._.coref_clusters)
#     else:
#         return ('', '')

In [10]:
def get_pred_list(pred_str):
    pred_list = []
    for line in pred_str.split('\n'):
        if 'gender' not in line:
            pred_list.append(line)
    return pred_list

In [11]:
def get_pred_equivalence(pred1, pred2):
    pred1_list = get_pred_list(str(pred1))
    pred2_list = get_pred_list(str(pred2))
#     print(pred1_list)
#     print(pred2_list)
    return pred1_list == pred2_list

In [12]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [13]:
def generate_sentences(oc1, oc2, verb, action, pronoun):


    aux_verb = random.choice(auxiliary_verb)
    input1 = ("The " + oc1 + " was " + aux_verb + ". " + pronoun[0] + " " 
              + verb + " " + "the " + oc2 + " " + filler_conjunction[verb] 
              + action + '.') 

    input2 = ("The " + oc1 + " was " + aux_verb + ". " + pronoun[1] + " " 
              + verb + " " + "the " + oc2 + " " + filler_conjunction[verb] 
              + action + '.') 
        
    return input1, input2
    

In [14]:
with open('saved_pickles/Exploration/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [15]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [16]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [17]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [18]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [19]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

occupation_pair_error_exploitation = {}

occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

occupation_pair_count_exploitation = {}

occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [20]:
oc1_probability = get_probability_dict(occupation1_error, occupation1_count)

print(oc1_probability)
print()

error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
print(error_rate_dict)
print()

output_dict = {}
for i in range(100000):
    oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'technician': 0.805964295571082, 'manager': 0.06766120012201676, 'developer': 0.06667344537570993, 'driver': 0.05970105893119126}

{'technician': 0.08823529411764706, 'manager': 0.007407407407407408, 'developer': 0.0072992700729927005, 'driver': 0.006535947712418301}

{'technician': 80415, 'manager': 6943, 'developer': 6632, 'driver': 6010}


In [21]:
STABILITY_ERRS = []
STABILITY_TEST_CASES = []

STABILITY_ITERS = 10

In [22]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    oc1_probability = get_probability_dict(occupation1_error, occupation1_count)
    oc2_probability = get_probability_dict(occupation2_error, occupation2_count)

    for i in range(ITERS):
#         oc1 = random.choice(occupations_1)
#         oc2 = random.choice(occupations_2)
        oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
        oc2 = get_weighted_random_choice(occupation2_error, occupation2_count, probablilities_dict=oc2_probability)
        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        input1, input2 = generate_sentences(oc1, oc2, verb, action, pronoun)


#         input3 = ("The " + oc1 + " " + verb + " "
#                + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        pred1 = predict_clusters(input1)
        pred2 = predict_clusters(input2)
#         pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if input1 not in unique_input1_set:
            unique_input1_set_exploitation.add(input1)

        update_dict(occupation_pair_count_exploitation, (oc1, oc2))
        update_dict(occupation1_count_exploitation, oc1)
        update_dict(occupation2_count_exploitation, oc2)
        update_dict(verb_count_exploitation, verb)
        update_dict(action_count_exploitation, action)



        if not (get_pred_equivalence(pred1, pred2)):
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if input1 not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add(input1)
                    
#                         if (pred2 != ''):
#                         print(pred1, pred2)
#                         print()
#                         print(input1)
#                         print(input2)
#                         print("--------------")
#                         else:
#                             print("empty pred2 error")
    #                 print(input3)

                    update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                    update_dict(occupation1_error_exploitation, oc1)
                    update_dict(occupation2_error_exploitation, oc2)
                    update_dict(verb_error_exploitation, verb)
                    update_dict(action_error_exploitation, action)



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))
    STABILITY_ERRS.append(unique_input1_error_set_exploitation)
    STABILITY_TEST_CASES.append(unique_input1_set_exploitation)

In [29]:
for i in range(STABILITY_ITERS):
    unique_input1_set_exploitation = set()
    unique_input1_error_set_exploitation = set()

    occupation_pair_error_exploitation = {}

    occupation1_error_exploitation = {}

    occupation2_error_exploitation = {}

    verb_error_exploitation = {}

    action_error_exploitation = {}

    occupation_pair_count_exploitation = {}

    occupation1_count_exploitation = {}

    occupation2_count_exploitation = {}

    verb_count_exploitation = {}

    action_count_exploitation = {}

    generate_test_sentences(ITERS=3000)

Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 4
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 7
Unique inputs: 59
Iterations: 60
------------------------------
Unique errors: 10
Unique inputs: 88
Iterations: 90
------------------------------
Unique errors: 13
Unique inputs: 115
Iterations: 120
------------------------------
Unique errors: 17
Unique inputs: 145
Iterations: 150
------------------------------
Unique errors: 21
Unique inputs: 174
Iterations: 180
------------------------------
Unique errors: 24
Unique inputs: 204
Iterations: 210
------------------------------
Unique errors: 26
Unique inputs: 233
Iterations: 240
------------------------------
Unique errors: 30
Unique inputs: 263
Iterations: 270
------------------------------
Unique errors: 35
Unique inputs: 292
Iterations: 300
------------------------------
Unique errors: 36
Unique inputs: 316
Iterations: 330
------------------------------
Uniqu

Unique errors: 233
Unique inputs: 2277
Iterations: 2880
------------------------------
Unique errors: 234
Unique inputs: 2292
Iterations: 2910
------------------------------
Unique errors: 235
Unique inputs: 2311
Iterations: 2940
------------------------------
Unique errors: 236
Unique inputs: 2330
Iterations: 2970
------------------------------
323
0.10766666666666666
Final Unique errors: 237
Final Unique inputs: 2351
Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 2
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 10
Unique inputs: 58
Iterations: 60
------------------------------
Unique errors: 14
Unique inputs: 87
Iterations: 90
------------------------------
Unique errors: 19
Unique inputs: 117
Iterations: 120
------------------------------
Unique errors: 21
Unique inputs: 146
Iterations: 150
------------------------------
Unique errors: 23
Unique inputs: 174
Iterations: 180
------------------------------
U

Unique errors: 251
Unique inputs: 2181
Iterations: 2730
------------------------------
Unique errors: 254
Unique inputs: 2200
Iterations: 2760
------------------------------
Unique errors: 255
Unique inputs: 2218
Iterations: 2790
------------------------------
Unique errors: 256
Unique inputs: 2236
Iterations: 2820
------------------------------
Unique errors: 258
Unique inputs: 2259
Iterations: 2850
------------------------------
Unique errors: 258
Unique inputs: 2279
Iterations: 2880
------------------------------
Unique errors: 261
Unique inputs: 2300
Iterations: 2910
------------------------------
Unique errors: 262
Unique inputs: 2319
Iterations: 2940
------------------------------
Unique errors: 263
Unique inputs: 2340
Iterations: 2970
------------------------------
358
0.11933333333333333
Final Unique errors: 266
Final Unique inputs: 2358
Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 6
Unique inputs: 30
Iterations: 30
-------------

Unique errors: 204
Unique inputs: 2090
Iterations: 2580
------------------------------
Unique errors: 204
Unique inputs: 2108
Iterations: 2610
------------------------------
Unique errors: 206
Unique inputs: 2127
Iterations: 2640
------------------------------
Unique errors: 209
Unique inputs: 2145
Iterations: 2670
------------------------------
Unique errors: 211
Unique inputs: 2164
Iterations: 2700
------------------------------
Unique errors: 213
Unique inputs: 2184
Iterations: 2730
------------------------------
Unique errors: 215
Unique inputs: 2203
Iterations: 2760
------------------------------
Unique errors: 217
Unique inputs: 2221
Iterations: 2790
------------------------------
Unique errors: 219
Unique inputs: 2243
Iterations: 2820
------------------------------
Unique errors: 219
Unique inputs: 2261
Iterations: 2850
------------------------------
Unique errors: 221
Unique inputs: 2283
Iterations: 2880
------------------------------
Unique errors: 223
Unique inputs: 2301
Iter

Unique errors: 189
Unique inputs: 1994
Iterations: 2430
------------------------------
Unique errors: 194
Unique inputs: 2014
Iterations: 2460
------------------------------
Unique errors: 195
Unique inputs: 2037
Iterations: 2490
------------------------------
Unique errors: 198
Unique inputs: 2057
Iterations: 2520
------------------------------
Unique errors: 199
Unique inputs: 2076
Iterations: 2550
------------------------------
Unique errors: 202
Unique inputs: 2095
Iterations: 2580
------------------------------
Unique errors: 204
Unique inputs: 2111
Iterations: 2610
------------------------------
Unique errors: 205
Unique inputs: 2129
Iterations: 2640
------------------------------
Unique errors: 209
Unique inputs: 2147
Iterations: 2670
------------------------------
Unique errors: 211
Unique inputs: 2170
Iterations: 2700
------------------------------
Unique errors: 212
Unique inputs: 2188
Iterations: 2730
------------------------------
Unique errors: 212
Unique inputs: 2209
Iter

...

Unique errors: 199
Unique inputs: 1787
Iterations: 2130
------------------------------
Unique errors: 203
Unique inputs: 1807
Iterations: 2160
------------------------------
Unique errors: 207
Unique inputs: 1832
Iterations: 2190
------------------------------
Unique errors: 209
Unique inputs: 1850
Iterations: 2220
------------------------------
Unique errors: 211
Unique inputs: 1872
Iterations: 2250
------------------------------
Unique errors: 213
Unique inputs: 1893
Iterations: 2280
------------------------------
Unique errors: 213
Unique inputs: 1912
Iterations: 2310
------------------------------
Unique errors: 214
Unique inputs: 1933
Iterations: 2340
------------------------------
Unique errors: 217
Unique inputs: 1951
Iterations: 2370
------------------------------
Unique errors: 220
Unique inputs: 1976
Iterations: 2400
------------------------------
Unique errors: 222
Unique inputs: 2001
Iterations: 2430
------------------------------
Unique errors: 226
Unique inputs: 2022
Iter

Unique errors: 192
Unique inputs: 1685
Iterations: 1980
------------------------------
Unique errors: 195
Unique inputs: 1702
Iterations: 2010
------------------------------
Unique errors: 198
Unique inputs: 1721
Iterations: 2040
------------------------------
Unique errors: 199
Unique inputs: 1740
Iterations: 2070
------------------------------
Unique errors: 201
Unique inputs: 1759
Iterations: 2100
------------------------------
Unique errors: 203
Unique inputs: 1780
Iterations: 2130
------------------------------
Unique errors: 204
Unique inputs: 1801
Iterations: 2160
------------------------------
Unique errors: 204
Unique inputs: 1822
Iterations: 2190
------------------------------
Unique errors: 209
Unique inputs: 1847
Iterations: 2220
------------------------------
Unique errors: 212
Unique inputs: 1869
Iterations: 2250
------------------------------
Unique errors: 214
Unique inputs: 1892
Iterations: 2280
------------------------------
Unique errors: 215
Unique inputs: 1912
Iter

Unique errors: 158
Unique inputs: 1562
Iterations: 1830
------------------------------
Unique errors: 159
Unique inputs: 1582
Iterations: 1860
------------------------------
Unique errors: 161
Unique inputs: 1604
Iterations: 1890
------------------------------
Unique errors: 164
Unique inputs: 1627
Iterations: 1920
------------------------------
Unique errors: 164
Unique inputs: 1648
Iterations: 1950
------------------------------
Unique errors: 167
Unique inputs: 1670
Iterations: 1980
------------------------------
Unique errors: 171
Unique inputs: 1689
Iterations: 2010
------------------------------
Unique errors: 173
Unique inputs: 1711
Iterations: 2040
------------------------------
Unique errors: 174
Unique inputs: 1730
Iterations: 2070
------------------------------
Unique errors: 178
Unique inputs: 1749
Iterations: 2100
------------------------------
Unique errors: 181
Unique inputs: 1769
Iterations: 2130
------------------------------
Unique errors: 181
Unique inputs: 1792
Iter

Unique errors: 166
Unique inputs: 1475
Iterations: 1680
------------------------------
Unique errors: 170
Unique inputs: 1498
Iterations: 1710
------------------------------
Unique errors: 171
Unique inputs: 1525
Iterations: 1740
------------------------------
Unique errors: 174
Unique inputs: 1547
Iterations: 1770
------------------------------
Unique errors: 175
Unique inputs: 1568
Iterations: 1800
------------------------------
Unique errors: 177
Unique inputs: 1591
Iterations: 1830
------------------------------
Unique errors: 179
Unique inputs: 1609
Iterations: 1860
------------------------------
Unique errors: 181
Unique inputs: 1629
Iterations: 1890
------------------------------
Unique errors: 182
Unique inputs: 1648
Iterations: 1920
------------------------------
Unique errors: 182
Unique inputs: 1671
Iterations: 1950
------------------------------
Unique errors: 183
Unique inputs: 1689
Iterations: 1980
------------------------------
Unique errors: 185
Unique inputs: 1713
Iter

Unique errors: 125
Unique inputs: 1338
Iterations: 1530
------------------------------
Unique errors: 131
Unique inputs: 1362
Iterations: 1560
------------------------------
Unique errors: 136
Unique inputs: 1389
Iterations: 1590
------------------------------
Unique errors: 140
Unique inputs: 1411
Iterations: 1620
------------------------------
Unique errors: 143
Unique inputs: 1435
Iterations: 1650
------------------------------
Unique errors: 147
Unique inputs: 1457
Iterations: 1680
------------------------------
Unique errors: 147
Unique inputs: 1479
Iterations: 1710
------------------------------
Unique errors: 149
Unique inputs: 1501
Iterations: 1740
------------------------------
Unique errors: 150
Unique inputs: 1527
Iterations: 1770
------------------------------
Unique errors: 151
Unique inputs: 1547
Iterations: 1800
------------------------------
Unique errors: 153
Unique inputs: 1570
Iterations: 1830
------------------------------
Unique errors: 156
Unique inputs: 1597
Iter

In [30]:
for i, err_set in enumerate(STABILITY_ERRS):
    print(len(err_set), len(STABILITY_TEST_CASES[i]))

1 10
1 10
1 10
0 10
1 10
0 10
2 10
1 9
0 10
2 10
237 2351
266 2358
226 2355
224 2357
253 2379
251 2369
262 2357
242 2348
253 2410
237 2372


In [25]:
# import pickle

# with open('saved_pickles/Exploitation/unique_input1_set.pickle', 'wb') as handle:
#     pickle.dump(unique_input1_set_exploitation, handle)
    
# with open('saved_pickles/Exploitation/unique_input1_error_set.pickle', 'wb') as handle:
#     pickle.dump(unique_input1_error_set_exploitation, handle)

In [26]:
# with open('saved_pickles/Exploitation/occupation_pair_count.pickle', 'wb') as handle:
#     pickle.dump(occupation_pair_count_exploitation, handle)
    
# with open('saved_pickles/Exploitation/occupation1_count.pickle', 'wb') as handle:
#     pickle.dump(occupation1_count_exploitation, handle)
    
# with open('saved_pickles/Exploitation/occupation2_count.pickle', 'wb') as handle:
#     pickle.dump(occupation2_count_exploitation, handle)
    
# with open('saved_pickles/Exploitation/verb_count.pickle', 'wb') as handle:
#     pickle.dump(verb_count_exploitation, handle)

# with open('saved_pickles/Exploitation/action_count.pickle', 'wb') as handle:
#     pickle.dump(action_count_exploitation, handle)

In [27]:
# with open('saved_pickles/Exploitation/occupation_pair_error.pickle', 'wb') as handle:
#     pickle.dump(occupation_pair_error_exploitation, handle)
    
# with open('saved_pickles/Exploitation/occupation1_error.pickle', 'wb') as handle:
#     pickle.dump(occupation1_error_exploitation, handle)
    
# with open('saved_pickles/Exploitation/occupation2_error.pickle', 'wb') as handle:
#     pickle.dump(occupation2_error_exploitation, handle)
    
# with open('saved_pickles/Exploitation/verb_error.pickle', 'wb') as handle:
#     pickle.dump(verb_error_exploitation, handle)

# with open('saved_pickles/Exploitation/action_error.pickle', 'wb') as handle:
#     pickle.dump(action_error_exploitation, handle)

In [28]:
# print(list(unique_input1_error_set_exploitation)[:20])